# BRIEF
IT NOW CAN PERFORM REGRESSION AND multi class CLASSIFICATION



In [1]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import OneHotEncoder

# Load Fashion MNIST dataset using Scikit-Learn
fashion_mnist = fetch_openml('Fashion-MNIST', version=1, as_frame=False)
images, labels = fashion_mnist["data"], fashion_mnist["target"].astype(int)

# Normalize pixel values to the range [0, 1]
images = images.astype('float32') / 255.0

# Reshape images from (num_samples, 784) to (num_samples, 1, 28, 28)
images = images.reshape((-1, 28*28))  # Convert to 4D for Conv2D: [B, C, H, W]

# Split into train and test sets
train_images_2d, test_images_2d = images[:60000], images[60000:]
train_labels_2d, test_labels_2d = labels[:60000], labels[60000:]

# One-hot encode the labels
encoder = OneHotEncoder()
train_labels_2d = encoder.fit_transform(train_labels_2d.reshape(-1, 1)).toarray()
test_labels_2d = encoder.transform(test_labels_2d.reshape(-1, 1)).toarray()

# Print shapes to verify
print("Train images shape:", train_images_2d.shape)  # (60000, 1, 28, 28)
print("Train labels shape:", train_labels_2d.shape)  # (60000, 10)
print("Test images shape:", test_images_2d.shape)    # (10000, 1, 28, 28)
print("Test labels shape:", test_labels_2d.shape)    # (10000, 10)


Train images shape: (60000, 784)
Train labels shape: (60000, 10)
Test images shape: (10000, 784)
Test labels shape: (10000, 10)


In [4]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import OneHotEncoder

# Load Fashion MNIST dataset using Scikit-Learn
fashion_mnist = fetch_openml('Fashion-MNIST', version=1, as_frame=False)
images, labels = fashion_mnist["data"], fashion_mnist["target"].astype(int)

# Normalize pixel values to the range [0, 1]
images = images.astype('float32') / 255.0

# Reshape images from (num_samples, 784) to (num_samples, 1, 28, 28)
images = images.reshape((-1, 1, 28, 28))  # Convert to 4D for Conv2D: [B, C, H, W]

# Split into train and test sets
train_images, test_images = images[:60000], images[60000:]
train_labels, test_labels = labels[:60000], labels[60000:]

# One-hot encode the labels
encoder = OneHotEncoder()
train_labels = encoder.fit_transform(train_labels.reshape(-1, 1)).toarray()
test_labels = encoder.transform(test_labels.reshape(-1, 1)).toarray()

# Print shapes to verify
print("Train images shape:", train_images.shape)  # (60000, 1, 28, 28)
print("Train labels shape:", train_labels.shape)  # (60000, 10)
print("Test images shape:", test_images.shape)    # (10000, 1, 28, 28)
print("Test labels shape:", test_labels.shape)    # (10000, 10)


Train images shape: (60000, 1, 28, 28)
Train labels shape: (60000, 10)
Test images shape: (10000, 1, 28, 28)
Test labels shape: (10000, 10)


In [3]:
def gradient_check(model, X, Y, loss_fn, epsilon=1e-7, threshold=1e-5):
    """
    Performs gradient checking for a given model.
    """
    # Forward pass to compute initial loss
    model.forward(X)
    loss, grad_output = loss_fn(Y, model.last_out, axis=1)

    # Backward pass to get analytical gradients (without optimizer step)
    error_grad = grad_output
    for layer in reversed(model.layers):
        error_grad = layer.backward(error_grad)

    for layer in model.layers:
        if not hasattr(layer, 'weights') or layer.weights is None:
            continue

        W, grad_W = layer.weights, layer.grad_w

        # ------ Numerical Gradient for Weights (element-wise) ------
        numerical_grad_W = np.zeros_like(W)
        original_weights = W.copy()

        for idx in np.ndindex(W.shape):
            # Perturb weight at idx by +epsilon
            W[idx] += epsilon
            model.forward(X)
            loss_plus = loss_fn(Y, model.last_out, axis=1)[0]

            # Perturb weight at idx by -epsilon
            W[idx] = original_weights[idx] - epsilon
            model.forward(X)
            loss_minus = loss_fn(Y, model.last_out, axis=1)[0]

            # Compute numerical gradient for this weight
            numerical_grad_W[idx] = (loss_plus - loss_minus) / (2 * epsilon)
            # Restore original weight
            W[idx] = original_weights[idx]

        # Compare analytical and numerical gradients
        diff_W = np.linalg.norm(grad_W - numerical_grad_W) / (np.linalg.norm(grad_W) + np.linalg.norm(numerical_grad_W) + 1e-15)
        passed = diff_W < threshold
        print(f"Layer {layer.__class__.__name__} (Weights): Gradient Check {'PASSED' if passed else 'FAILED'} (diff: {diff_W:.8e})")
        # # if not passed:
        # print(f"Numerical Gradient: {numerical_grad_W}")
        # print(f"network Gradient: {grad_W}")
        # ------ Numerical Gradient for Biases (if present) ------
        if hasattr(layer, 'bias') and layer.bias is not None:
            b, grad_b = layer.bias, layer.grad_b
            numerical_grad_b = np.zeros_like(b)
            original_bias = b.copy()

            for idx in np.ndindex(b.shape):
                b[idx] += epsilon
                model.forward(X)
                loss_plus = loss_fn(Y, model.last_out, axis=1)[0]

                b[idx] = original_bias[idx] - epsilon
                model.forward(X)
                loss_minus = loss_fn(Y, model.last_out, axis=1)[0]

                numerical_grad_b[idx] = (loss_plus - loss_minus) / (2 * epsilon)
                b[idx] = original_bias[idx]

            diff_b = np.linalg.norm(grad_b - numerical_grad_b) / (np.linalg.norm(grad_b) + np.linalg.norm(numerical_grad_b) + 1e-15)
            passed = diff_b < threshold
            print(f"Layer {layer.__class__.__name__} (Biases): Gradient Check {'PASSED' if passed else 'FAILED'} (diff: {diff_b:.8e})\n ")
            # if not passed:
            # print(f"Numerical Gradient: {numerical_grad_b}")
            # print(f"network Gradient: {grad_b}")
                





In [5]:
from core.nn import Linear, Network,batchnorm1d,Conv2d,MaxPool2d
from core.Function import tanh,relu,softmax
from core.loss import sparse_categorical_cross_entropy,mean_squared_error
from core.utils import  gradient_check
conv1 = Conv2d(input_channels=1, output_channels=8, kernel_size=3, stride=1, padding=1)
max1 = MaxPool2d(kernel_size=2, stride=2)
tanh1 = tanh()
conv2 = Conv2d(input_channels=8, output_channels=16, kernel_size=3, stride=1, padding=1)
tanh2 = tanh()
max2 = MaxPool2d(kernel_size=2, stride=2)
linear1 = Linear((16 * 7 * 7,50),activation=None)
tanh3 = tanh()
linear2 = Linear((50,10))
softmaxx = softmax()

layers = [conv1,tanh1,max1,conv2,tanh2,max2,linear1,tanh3,linear2,softmaxx]


model_coco = Network(layers,classification=True)


model_coco.train(train_images, train_labels, epochs=10, batch_size=64, learning_rate=0.001, optimizer="sgd",verbose=1)
#gradient_check(model_coco, test_images[:10], test_labels[:10], sparse_categorical_cross_entropy)

weights (784, 50)
bias (1, 50)
weights (50, 10)
bias (1, 10)
Epoch 1/10| Epoch Loss = 2.3003
Epoch 2/10| Epoch Loss = 2.2937
Epoch 3/10| Epoch Loss = 2.2794
Epoch 4/10| Epoch Loss = 2.2375
Epoch 5/10| Epoch Loss = 2.1029
Epoch 6/10| Epoch Loss = 1.8299
Epoch 7/10| Epoch Loss = 1.5713
Epoch 8/10| Epoch Loss = 1.4013
Epoch 9/10| Epoch Loss = 1.2831
Epoch 10/10| Epoch Loss = 1.1898

Function train took 716.4680 seconds to execute.


In [6]:
from core.utils import save_model
save_model(model_coco)

Model saved as model.pkl


In [ ]:
model_coco.visualize_feature_maps(test_images[1])

In [ ]:
model_coco.plot_loss()

In [ ]:
# Reshape test data for Conv2d input (example for MNIST)
test_images_4d = test_images.reshape(-1, 1, 28, 28)  # Adjust dimensions as needed

# Predict
y_pred = model_coco.predict(test_images_4d)
y_pred_classes = np.argmax(y_pred, axis=1, keepdims=True)

# Use integer labels directly (if test_labels are not one-hot)
y_true_classes = np.argmax(test_labels, axis=1, keepdims=True)  # Assuming test_labels are integers
print("ypred_shape",y_pred_classes.shape)
print("y_true_shape",y_true_classes.shape)
# Calculate accuracy
accuracy = np.mean(y_pred_classes == y_true_classes)
print(f"Accuracy on test set: {accuracy:.4f}")

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import time

conv_forward_time = []
conv_backward_time = []
pool_forward_time = []
pool_backward_time = []
# Define the model (same as your original model)
class FashionMNISTModel(nn.Module):
    def __init__(self):
        super(FashionMNISTModel, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.Tanh()
        self.max1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.Tanh()
        self.max2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(16 * 7 * 7, 50)  # Fully connected layer
        self.relu3 = nn.Tanh()
        self.fc2 = nn.Linear(50, 10)
        

    def forward(self, x):
       
        x = self.conv1(x)
      
        x = self.relu1(x)
      
        x = self.max1(x)
        
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.max2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.fc2(x)
        
        return x

# Transform for normalizing the data
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert image to PyTorch tensor
    transforms.Normalize((0.5,), (0.5,))  # Normalize with mean 0.5 and std 0.5
])

# Load the FashionMNIST dataset from torchvision
trainset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

# Create DataLoader for training and test datasets
train_loader = DataLoader(trainset, batch_size=64, shuffle=True)
test_loader = DataLoader(testset, batch_size=64, shuffle=False)

# Instantiate the model, loss function, and optimizer
model = FashionMNISTModel()
criterion = nn.CrossEntropyLoss()  # Use raw class labels for CrossEntropyLoss
optimizer = optim.SGD(model.parameters(), lr=0.001)
forward_time = []
backward_time = []
# Training loop
def train_model(model, train_loader, epochs, optimizer, criterion):
    model.train()  # Set model to training mode
    for epoch in range(1, epochs + 1):
        epoch_loss = 0.0
        for images, labels in train_loader:
            optimizer.zero_grad()  # Zero gradients
          
            outputs = model(images)
        
            
            loss = criterion(outputs, labels)  # Labels are in class indices format
          
            loss.backward()
        
            optimizer.step()
            epoch_loss += loss.item()

        avg_loss = epoch_loss / len(train_loader)
        print(f'Epoch {epoch}/{epochs}, Loss: {avg_loss:.4f}')

# Testing loop
def test_model(model, test_loader):
    model.eval()  # Set model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():  # Disable gradient calculation for testing
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)  # Get the class with the highest probability
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy:.2f}%')

# Train the model
train_model(model, train_loader, epochs=10, optimizer=optimizer, criterion=criterion)

# Test the model on test data
test_model(model, test_loader)
